In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from scipy.optimize import curve_fit
import glob
from astropy.table import Table, vstack
import pandas as pd
import statsmodels.api as sm
import pandas as pd
from astropy.table import Table, vstack
import glob as glob
from datetime import datetime
from collections import OrderedDict

size_outside_cut = 500
muon_efficiency_cut = 1
min_impact = 2.2199933748101555
max_impact = 9.983608702234397


In [ ]:
def read_and_process_fits(files):
    dat = None
    for muon_file in files:
        dat2 = Table.read(muon_file, format='fits')
        
        # Convert columns to boolean if they exist
        for col in ['good_ring', 'is_valid', 'parameters_at_limit']:
            if col in dat2.colnames:
                dat2[col] = dat2[col].astype(bool)
        
        # Stack the tables
        dat = vstack([dat, dat2]) if dat is not None else dat2
    
    return dat

# Define a function to filter the DataFrame
def filter_dataframe(df, muon_efficiency_cut, size_outside_cut):
    return df[
        df['good_ring'] & 
        (df['muon_efficiency'] < muon_efficiency_cut) & 
        (df['size_outside'] < size_outside_cut)
    ]

In [ ]:
# Process the first set of files
listdir = glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/nsbtune2024year_tuned_reflectivity_00025alignment/tables/*')
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_sim_00025_align = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)


listdir1 = glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/nsbtune2024year_tuned_reflectivity/tables/*')
listdir2 = glob.glob('/Users/vdk/muons2024/lapalma_simulations/for_paper/nsbtune2024year_no_outliers_additional_true/tables/*')
listdir = listdir1 + listdir2
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_sim_00046_align = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)


listdir = glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/psf_sim/nsbtune2024year_tuned_reflectivity_00092alignment/tables/*')
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_sim_0092_align = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)


listdir = glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/nsbtune2024year_tuned_reflectivity_0015alignment/tables/*')
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_sim_0015_align = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)

listdir= glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/psf_sim/nsbtune2024year_tuned_reflectivity_002alignment/tables/*')
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_sim_002_align = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)

In [ ]:
listdir = glob.glob('/Users/vdk/muons2024/data_quality_winter_2024/tables/*')
dat = read_and_process_fits(listdir)
df = dat.to_pandas()
df_winter_data = filter_dataframe(df, muon_efficiency_cut, size_outside_cut)

start_date_2024 = datetime.strptime("2024-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2024 = datetime.strptime("2024-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

muon_files = glob.glob('/Users/vdk/Documents/DocumentsVadymMacBookWork/all_muon_fits/low_nsb_filter/*')
for i, filename in enumerate(sorted(muon_files)):
    print(i, filename)

df_files = [
    pd.read_csv(muon_file, na_values=['NA', '?']).query(
        '(muon_efficiency < @muon_efficiency_cut) & '
        '(size_outside < @size_outside_cut) & '
        '(event_time >= @start_date_2024) & '
        '(event_time <= @end_date_2024)'
    )
    for muon_file in sorted(muon_files)[10:]
]

df_good_data_real = pd.concat(df_files, ignore_index=True)
df_good_data_real

In [ ]:
ring_completeness_cut = 0.9
ring_containment_cut = 0
min_radius_cut = 0
max_radius_cut = 2
ring_center_distance_cut = 2
impact_distance_cut = 20

def calculate_survival_percentage(df, df_cut):
    return (len(df_cut) / len(df)) * 100

def apply_cuts(df, completeness_cut, containment_cut, min_radius_cut, max_radius_cut, center_distance_cut, impact_distance_cut):
    return df[
        (df['ring_completeness'] > completeness_cut) &
        (df['ring_containment'] > containment_cut) &
        (df['ring_radius'] > min_radius_cut) &
        (df['ring_radius'] < max_radius_cut) &
        (df['ring_center_x'].abs() < center_distance_cut) &
        (df['ring_center_y'].abs() < center_distance_cut) &
        (df['impact_parameter'] < impact_distance_cut)
    ]


df_sim_00025_cut = apply_cuts(df_sim_00025_align, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_sim_00025_cut: {len(df_sim_00025_cut)} ({calculate_survival_percentage(df_sim_00025_align, df_sim_00025_cut):.2f}%)")

df_sim_00046_cut = apply_cuts(df_sim_00046_align, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_sim_00043_cut: {len(df_sim_00046_cut)} ({calculate_survival_percentage(df_sim_00046_align, df_sim_00046_cut):.2f}%)")

df_sim_0092_cut = apply_cuts(df_sim_0092_align, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_sim_0092_cut: {len(df_sim_0092_cut)} ({calculate_survival_percentage(df_sim_0092_align, df_sim_0092_cut):.2f}%)")

df_sim_0015_cut = apply_cuts(df_sim_0015_align, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_sim_0015_cut: {len(df_sim_0015_cut)} ({calculate_survival_percentage(df_sim_0015_align, df_sim_0015_cut):.2f}%)")

df_sim_002_cut = apply_cuts(df_sim_002_align, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_sim_002_cut: {len(df_sim_002_cut)} ({calculate_survival_percentage(df_sim_002_align, df_sim_002_cut):.2f}%)")

df_data_cut = apply_cuts(df_good_data_real, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_data_cut: {len(df_data_cut)} ({calculate_survival_percentage(df_good_data_real, df_data_cut):.2f}%)")

df_winter_2024_cut = apply_cuts(df_winter_data, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_winter_2024_cut: {len(df_winter_2024_cut)} ({calculate_survival_percentage(df_winter_data, df_winter_2024_cut):.2f}%)")

In [ ]:
print(f"Winter data ring radius min and max: {min(df_winter_data['ring_radius']):.3f}, {max(df_winter_data['ring_radius']):.3f}")
print(f"For 0.0046 deg radius min and max = {min(df_sim_00046_align['ring_radius']):.3f} and {max(df_sim_00046_align['ring_radius']):.3f}")
print(f"For 0.0092 deg radius min and max = {min(df_sim_0092_align['ring_radius']):.3f} and {max(df_sim_0092_align['ring_radius']):.3f}")
print(f"For 0.015 deg radius min and max = {min(df_sim_0015_align['ring_radius']):.3f} and {max(df_sim_0015_align['ring_radius']):.3f}")

In [ ]:
def apply_radius_cut(df, min_radius_cut, max_radius_cut):
    return df[
        (df['ring_radius'] > min_radius_cut) &
        (df['ring_radius'] < max_radius_cut)
    ]

In [ ]:
min_radius_cut = 0.91
max_radius_cut = 1.23

df_sim_00025_radius_cut = apply_radius_cut(df_sim_00025_cut, min_radius_cut, max_radius_cut)
df_sim_00046_radius_cut = apply_radius_cut(df_sim_00046_cut, min_radius_cut, max_radius_cut)
df_sim_0092_radius_cut = apply_radius_cut(df_sim_0092_cut, min_radius_cut, max_radius_cut)
df_sim_0015_radius_cut = apply_radius_cut(df_sim_0015_cut, min_radius_cut, max_radius_cut)
df_sim_002_radius_cut = apply_radius_cut(df_sim_002_cut, min_radius_cut, max_radius_cut)

df_winter_2024_radius_cut = apply_radius_cut(df_winter_2024_cut, min_radius_cut, max_radius_cut)
df_data_cut_radius_cut = apply_radius_cut(df_data_cut, min_radius_cut, max_radius_cut)

print(f"2024 Winter Data ring radius min and max: {min(df_winter_2024_radius_cut['ring_radius']):.3f}, {max(df_winter_2024_radius_cut['ring_radius']):.3f}")
print(f"All 2024 Data ring radius min and max: {min(df_data_cut_radius_cut['ring_radius']):.3f}, {max(df_data_cut_radius_cut['ring_radius']):.3f}")

print(f"For 0.0025 deg radius min and max = {min(df_sim_00025_radius_cut['ring_radius']):.3f} and {max(df_sim_00025_radius_cut['ring_radius']):.3f}")
print(f"For 0.0046 deg radius min and max = {min(df_sim_00046_radius_cut['ring_radius']):.3f} and {max(df_sim_00046_radius_cut['ring_radius']):.3f}")
print(f"For 0.0092 deg radius min and max = {min(df_sim_0092_radius_cut['ring_radius']):.3f} and {max(df_sim_0092_radius_cut['ring_radius']):.3f}")
print(f"For 0.015 deg radius min and max = {min(df_sim_0015_radius_cut['ring_radius']):.3f} and {max(df_sim_0015_radius_cut['ring_radius']):.3f}")
print(f"For 0.002 deg radius min and max = {min(df_sim_002_radius_cut['ring_radius']):.3f} and {max(df_sim_002_radius_cut['ring_radius']):.3f}")

In [ ]:
bin_edges = max_radius_cut - min_radius_cut
bin_bins = np.linspace(min_radius_cut, max_radius_cut, 10)
bin_centers = (bin_bins[1:] + bin_bins[:-1]) / 2

In [ ]:
df_winter_2024_radius_cut['radius_bin'] = pd.cut(df_winter_2024_radius_cut['ring_radius'], bins=bin_bins, include_lowest=True)
df_data_cut_radius_cut['radius_bin'] = pd.cut(df_data_cut_radius_cut['ring_radius'], bins=bin_bins, include_lowest=True)

df_sim_00025_radius_cut['radius_bin'] = pd.cut(df_sim_00025_radius_cut['ring_radius'], bins=bin_bins, include_lowest=True)
df_sim_00046_radius_cut['radius_bin'] = pd.cut(df_sim_00046_radius_cut['ring_radius'], bins=bin_bins, include_lowest=True)
df_sim_0092_radius_cut['radius_bin'] = pd.cut(df_sim_0092_radius_cut['ring_radius'], bins=bin_bins, include_lowest=True)
df_sim_0015_radius_cut['radius_bin'] = pd.cut(df_sim_0015_radius_cut['ring_radius'], bins=bin_bins, include_lowest=True)
df_sim_002_radius_cut['radius_bin'] = pd.cut(df_sim_002_radius_cut['ring_radius'], bins=bin_bins, include_lowest=True)

In [ ]:
data_ring_width_binned = df_winter_2024_radius_cut.groupby('radius_bin')['ring_width'].agg(['mean', 'sum', 'count']).reset_index()
all_data_ring_width_binned = df_data_cut_radius_cut.groupby('radius_bin')['ring_width'].agg(['mean', 'sum', 'count']).reset_index()

sim_00025_width_binned = df_sim_00025_radius_cut.groupby('radius_bin')['ring_width'].agg(['mean', 'sum', 'count']).reset_index()
sim_00046_width_binned = df_sim_00046_radius_cut.groupby('radius_bin')['ring_width'].agg(['mean', 'sum', 'count']).reset_index()
sim_0092_width_binned = df_sim_0092_radius_cut.groupby('radius_bin')['ring_width'].agg(['mean', 'sum', 'count']).reset_index()
sim_0015_width_binned = df_sim_0015_radius_cut.groupby('radius_bin')['ring_width'].agg(['mean', 'sum', 'count']).reset_index()
sim_002_width_binned = df_sim_002_radius_cut.groupby('radius_bin')['ring_width'].agg(['mean', 'sum', 'count']).reset_index()

In [ ]:
data_ring_width_binned['error'] = data_ring_width_binned['mean'] / np.sqrt(data_ring_width_binned['count'])
all_data_ring_width_binned['error'] = all_data_ring_width_binned['mean'] / np.sqrt(all_data_ring_width_binned['count'])

sim_00025_width_binned['error'] = sim_00025_width_binned['mean'] / np.sqrt(sim_00025_width_binned['count'])
sim_00046_width_binned['error'] = sim_00046_width_binned['mean'] / np.sqrt(sim_00046_width_binned['count'])
sim_0092_width_binned['error'] = sim_0092_width_binned['mean'] / np.sqrt(sim_0092_width_binned['count'])
sim_0015_width_binned['error'] = sim_0015_width_binned['mean'] / np.sqrt(sim_0015_width_binned['count'])
sim_002_width_binned['error'] = sim_002_width_binned['mean'] / np.sqrt(sim_002_width_binned['count'])

In [ ]:
plt.figure(figsize=(11, 7.5)) 

plt.errorbar(
    bin_centers,
    data_ring_width_binned['mean'] / bin_centers,
    yerr=data_ring_width_binned['error'] / bin_centers,
    fmt='o',
    label='Good Data Winter 2024',
    color='gray',
    capsize=5  # Adds caps to the error bars
)

plt.errorbar(
    bin_centers,
    all_data_ring_width_binned['mean'] / bin_centers,
    yerr=all_data_ring_width_binned['error'] / bin_centers,
    fmt='o',
    label='All Data 2024',
    color='black',
    capsize=5
)

plt.errorbar(
    bin_centers,
    sim_00025_width_binned['mean'] / bin_centers,
    yerr=sim_00025_width_binned['error'] / bin_centers,
    fmt='o',
    label='MC 0.0025',
    color='purple',
    capsize=5
)

plt.errorbar(
    bin_centers,
    sim_00046_width_binned['mean'] / bin_centers,
    yerr=sim_00046_width_binned['error'] / bin_centers,
    fmt='o',
    label='MC 0.0046',
    color='orange',
    capsize=5
)


plt.errorbar(
    bin_centers,
    sim_0092_width_binned['mean'] / bin_centers,
    yerr=sim_0092_width_binned['error'] / bin_centers,
    fmt='o',
    label='MC 0.0092',
    color='green',
    capsize=5
)


plt.errorbar(
    bin_centers,
    sim_0015_width_binned['mean'] / bin_centers,
    yerr=sim_0015_width_binned['error'] / bin_centers,
    fmt='o',
    label='MC 0.015',
    color='red',
    capsize=5
)

plt.errorbar(
    bin_centers,
    sim_002_width_binned['mean'] / bin_centers,
    yerr=sim_002_width_binned['error'] / bin_centers,
    fmt='o',
    label='MC 0.02',
    color='blue',
    capsize=5
)


plt.xlabel('Ring Radius (bin centers)')
plt.ylabel('Mean Ring Width / Radius')
plt.title('Mean Ring Width per Radius Bin with Statistical Errors')
plt.legend()
plt.grid(True)
plt.tight_layout()  # Adjusts subplot params for better layout
plt.show()

# Regression line

In [ ]:
# Organize datasets into a list of dictionaries
datasets = [
    {'df': data_ring_width_binned, 'label': 'Data', 'color': 'gray'},
    {'df': all_data_ring_width_binned, 'label': 'All Data 2024 year', 'color': 'black'},
    {'df': sim_00025_width_binned, 'label': 'MC 0.0025', 'color': 'purple'},
    {'df': sim_00046_width_binned, 'label': 'MC 0.0046', 'color': 'orange'},
    {'df': sim_0092_width_binned, 'label': 'MC 0.0092', 'color': 'green'},
    {'df': sim_0015_width_binned, 'label': 'MC 0.015', 'color': 'red'},
    {'df': sim_002_width_binned, 'label': 'MC 0.02', 'color': 'blue'}
]

datasets = [
    {'df': all_data_ring_width_binned, 'label': 'All Data 2024 year', 'color': 'black'},
    {'df': sim_00025_width_binned, 'label': 'MC 0.0025', 'color': 'purple'},
    {'df': sim_00046_width_binned, 'label': 'MC 0.0046', 'color': 'orange'},
    {'df': sim_0092_width_binned, 'label': 'MC 0.0092', 'color': 'green'},
    {'df': sim_0015_width_binned, 'label': 'MC 0.015', 'color': 'red'},
    {'df': sim_002_width_binned, 'label': 'MC 0.02', 'color': 'blue'}
]

# Initialize the plot
plt.figure(figsize=(11, 7.5))

# Iterate over each dataset to plot error bars and regression lines
for dataset in datasets:
    df = dataset['df']
    label = dataset['label']
    color = dataset['color']
    
    # Extract x and y values
    x = bin_centers
    y = df['mean'] / x
    y_err = df['error'] / x
    
    # Plot data points with error bars
    plt.errorbar(
        x,
        y,
        yerr=y_err,
        fmt='o',
        label=label,
        color=color,
        capsize=5
    )
    
    # Perform linear regression (degree=1)
    slope, intercept = np.polyfit(x, y, 1)
    
    # Generate y-values for the regression line
    y_fit = slope * x + intercept
    
    # Plot the regression line
    plt.plot(x, y_fit, color=color, linestyle='--')

# Adjust legend to avoid duplicate entries
handles, labels = plt.gca().get_legend_handles_labels()
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

# Customize the plot
plt.xlabel('Ring Radius (bin centers)')
plt.ylabel('Mean Ring Width / Radius')
plt.ylim(0.01, 0.1)
plt.title('Mean Ring Width / Ring Radius per Radius Bin with Statistical Errors and Regression Lines')
plt.grid(alpha=0.3)
plt.tight_layout()  # Adjust layout for better spacing
plt.show()


# Points with displacement

In [ ]:
# Calculate Errors
data_ring_width_binned['error'] = data_ring_width_binned['mean'] / np.sqrt(data_ring_width_binned['count'])
sim_00046_width_binned['error'] = sim_00046_width_binned['mean'] / np.sqrt(sim_00046_width_binned['count'])
sim_0092_width_binned['error'] = sim_0092_width_binned['mean'] / np.sqrt(sim_0092_width_binned['count'])
sim_0015_width_binned['error'] = sim_0015_width_binned['mean'] / np.sqrt(sim_0015_width_binned['count'])

# Define Offsets
num_datasets = 4  # Data + 3 MC simulations
total_width = 0.01   # Total offset range
offset_step = total_width / (num_datasets - 1)  # e.g., 4 / 3 ≈ 1.333
offsets = np.linspace(-total_width / 2, total_width / 2, num_datasets)
# offsets = [-2.0, -0.666..., +0.666..., +2.0]

# Assign each dataset an offset
datasets = [
    {'df': data_ring_width_binned, 'label': 'Data', 'color': 'darkblue'},
    {'df': sim_00046_width_binned, 'label': 'MC 0.0046', 'color': 'orange'},
    {'df': sim_0092_width_binned, 'label': 'MC 0.0092', 'color': 'green'},
    {'df': sim_0015_width_binned, 'label': 'MC 0.015', 'color': 'red'}
]

plt.figure(figsize=(10, 6))

for dataset, offset in zip(datasets, offsets):
    shifted_bin_centers = bin_centers + offset  # Apply offset
    plt.errorbar(
        shifted_bin_centers,
        dataset['df']['mean'] / bin_centers,
        yerr=dataset['df']['error'] / bin_centers,
        fmt='o',
        label=dataset['label'],
        color=dataset['color'],
        capsize=5  # Adds caps to the error bars
    )

# Customize the Plot
plt.xlabel('Ring Radius (bin centers)')
plt.ylabel('Mean Ring Width / Radius')
plt.title('Mean Ring Width per Radius Bin with Statistical Errors')
plt.legend()
plt.grid(True)
#plt.xticks(bin_centers, bin_bins[:-1] + np.diff(bin_bins) / 2)  # Ensures x-ticks are at bin centers
plt.tight_layout()  # Adjusts subplot params for better layout
plt.show()